# Classification with Random Forest Classifier  

After cleaning and analysing the dataset we proceed with a regression, which is carried on with the random forest classifier algorithm

## Preparing the data 

In [9]:
![random-forest.jpg](attachment:ca783f13-594d-4e23-bc3b-8428378dffe3.jpg)

"[random-forest.jpg]" non è riconosciuto come comando interno o esterno,
 un programma eseguibile o un file batch.


We import the relevant libraries from ScikitLearn

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, roc_curve, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

We now preprocess the data. 
The classifier only really accepts numbers as an input, so string-to-number conversion of categorical data is essential.
Since this process only concerns categorical data,  we need to split numerical and categorical features first. 

In [5]:

categorical_features = ['date', 'municipality.name', 'hour_category']  # Any string or categorical features
numerical_features = ['temperature', 
                      'minTemperature', 
                      'maxTemperature', 
                      'precipitation', 
                      'tweet_count', 
                      'wind_speed',
                      'wind_dir',
                      'curr_cell',
                      'curr_site']

Second, we enconde the categorical data with label encoding. 

In [6]:
# Encode categorical features
le = LabelEncoder()
for feat in categorical_features:
    X[feat] = le.fit_transform(X[feat])

NameError: name 'LabelEncoder' is not defined

In [ ]:
X[categorical_features]

We then scale the numerical features, convert them back to dataframe form and finally recombine categorical and numerical data 

In [ ]:
# Scale the numerical features
scaler = StandardScaler()
scaled_numerical = scaler.fit_transform(X[numerical_features])


In [ ]:
# Scale the numerical features
scaler = StandardScaler()
scaled_numerical = scaler.fit_transform(X[numerical_features])

In [25]:

# Combine the scaled numerical features and encoded categorical features
X = pd.concat([scaled_numerical_df, X[categorical_features].astype('category')], axis=1)

NameError: name 'pd' is not defined

This is now a dataframe we can work with. 

In [26]:
X.dtypes

NameError: name 'X' is not defined

Now we prepare for the regression by splitting Train Data and Test Data. 
We want the test data to be the 20 % of the data available (test_size=0.2) and we want to fix a random_state value of 20. 

This is basically like a seed and assures the "random behaviour of the forest, to be always the same if we run the program the program multiple times. 

In the fase of tuning repeatability is essential, because it is the only way to reliably tune the parameters of the model. 

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

NameError: name 'train_test_split' is not defined

We initialise and train the Random Forest Classifier. 
We initialise it to a pretty canonical number of estimators (100) and to the already mentioned random_state=20.  
We don't use a max_widht parameter since the completion of the random forest's training is very fast.

In [36]:
# Initialize Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=20)

In [37]:
# Train the classifier
rf_classifier.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [31]:
Eventually, we obtain a prediction on the data 

SyntaxError: invalid syntax (3451178437.py, line 1)

In [32]:
y_pred = rf_classifier.predict(X_test)

NameError: name 'X_test' is not defined

In [34]:
We calculate here some meaningful extimators 

SyntaxError: invalid syntax (945106861.py, line 1)

In [35]:
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

NameError: name 'y_test' is not defined

We notice the accuracy is really close to 1, which indicates the goodnes of the described model. 
It is then a good idea to plot a confusion matrix and a heatmap to see what points have been misclassified.

In [38]:
confusion_matrix(y_test, y_pred)

ax= plt.subplot()
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(['high', 'low', 'medium'])
ax.yaxis.set_ticklabels(['high', 'low', 'medium'])

NameError: name 'y_test' is not defined

The remarkable fact is that just 1 point has been misclassified . 

This prediction is possible with a particular choice of the dataset. 

The only set of features that appears to be relevant is that related to the weather, and indeed the changing of the classifier's random_state variable does not alter this fact.

The strongest predictor for tweet count is arguably temperature, that is evident by seeing at the plot which is available in the EDA notebook.

We noticed that some features as month, day, day of the week and electrical data, actually worsen the model. More values are in fact mispredicted.
The error is always the same, some municipalities, which are predicted to be in the middle tweet count range, actually end up to be in the high tweet count range. 

This makes sense, because, whereas there are not many events outside our dataset which can cause great short term depression in the tweet count, there are lots of social events that may cause an occasional peak in the tweet count. 
In the dates 13 and 14 Decembers 2013, which present the greatest tweet predictive mistake, a big event called "Universiadi " was held in the municipality of Trento, where such peak is registered. 

If we insert those data, we have a bigger variance because we enlarge the model with data which is not significantly correlated.
We may be experimenting an overfitting phenomenon, increasing the features of the data requires increasing information to effectively model patterns without overfitting.
If the mentioned data are not higly informative, and our EDA results suggest precisely that, we can very well incur in this overfitting problem.  


